In [1]:
import torch


assert torch.cuda.is_available(), "CUDA NOT FOUND"

In [2]:
import os


os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [3]:
import pandas as pd


train = pd.read_csv('../data/TRAIN.csv').drop('Unnamed: 0', axis=1)
test = pd.read_csv('../data/TEST.csv').drop('Unnamed: 0', axis=1)
validation = pd.read_csv('../data/VALIDATION.csv').drop('Unnamed: 0', axis=1)

In [4]:
train['label'] = 1 - train['label']
test['label'] = 1 - test['label']
validation['label'] = 1 - validation['label']

In [5]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler


X = train['text'].to_frame()
y = train['label']

ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X, y)

train_balanced_oversampled = pd.concat([X_resampled, y_resampled], axis=1)
train_balanced_oversampled.columns = ['text', 'label']
train_balanced_oversampled.reset_index(inplace=True, drop='index')

rus = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = rus.fit_resample(X, y)

train_balanced_undersampled = pd.concat([X_resampled, y_resampled], axis=1)
train_balanced_undersampled.columns = ['text', 'label']
train_balanced_undersampled.reset_index(inplace=True, drop='index')

In [6]:
from datasets import (
    Dataset,
    DatasetDict
)


# oversampling works better
tds = Dataset.from_pandas(train_balanced_oversampled)
testds = Dataset.from_pandas(test)
vds = Dataset.from_pandas(validation)
dataset = DatasetDict()
dataset['train'] = tds
dataset['test'] = testds
dataset['validation'] = vds

In [7]:
from transformers import AutoTokenizer


model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    tokenized = tokenizer(batch["text"], padding=True, truncation=True)
    tokenized['label'] = batch['label']
    return tokenized

dataset_encoded = dataset.map(tokenize, batched=True, batch_size=64, remove_columns=['text'])

Map:   0%|          | 0/150382 [00:00<?, ? examples/s]

Map:   0%|          | 0/12396 [00:00<?, ? examples/s]

Map:   0%|          | 0/12395 [00:00<?, ? examples/s]

In [8]:
dataset_encoded.set_format("torch", columns=["input_ids", "attention_mask", "label"])
dataset_encoded.save_to_disk('../data/dataset_encoded_oversampled')

Saving the dataset (0/1 shards):   0%|          | 0/150382 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/12396 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/12395 [00:00<?, ? examples/s]

In [9]:
from transformers import DistilBertForSequenceClassification


model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
import numpy as np
from transformers import (
    TrainingArguments,
    Trainer
)


def train(dataset, model, training_args):

    def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        predictions = np.argmax(predictions, axis=1)

        metrics = {}
        metrics['accuracy'] = (predictions == labels).mean()

        tp = np.sum((predictions == 1) & (labels == 1))
        fp = np.sum((predictions == 1) & (labels == 0))
        fn = np.sum((predictions == 0) & (labels == 1))

        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        metrics['precision'] = precision
        metrics['recall'] = recall
        metrics['f1'] = f1

        return metrics

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset['train'],
        eval_dataset=dataset['validation'],
        compute_metrics=compute_metrics,
    )

    trainer.train()

    return trainer.model, tokenizer

In [11]:
training_args = TrainingArguments(
    output_dir="./aita_classifier",
    learning_rate=2e-5,
    lr_scheduler_type="cosine",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    eval_strategy="steps",
    eval_steps=200,
    save_strategy="steps",
    save_steps=200,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    push_to_hub=False,
    logging_steps=50,
    report_to=["tensorboard"],
    fp16=torch.cuda.is_available(),
)

In [12]:
model, tokenizer = train(dataset_encoded, model, training_args)

Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
200,0.660900,0.661676,0.592416,0.821622,0.584782,0.683260
400,0.659300,0.666527,0.582170,0.837657,0.550977,0.664725
600,0.654000,0.649619,0.620976,0.835074,0.617836,0.710215
800,0.629900,0.639475,0.620653,0.830944,0.621915,0.711392
1000,0.633900,0.626119,0.625494,0.838252,0.621807,0.713986
1200,0.629500,0.678063,0.571359,0.858293,0.514810,0.643590
1400,0.611300,0.658705,0.611214,0.854308,0.582099,0.692411
1600,0.603500,0.581458,0.679064,0.825292,0.726980,0.773023
1800,0.607100,0.605404,0.654296,0.836565,0.671281,0.744865
2000,0.582400,0.600975,0.666478,0.835838,0.692316,0.757337


In [36]:

def preprocess_text(text):
    """Clean and preprocess text"""
    # Convert to lowercase
    text = text.lower()
    # Remove multiple spaces
    text = ' '.join(text.split())
    # Remove special characters but keep punctuation
    text = ''.join([c if c.isalnum() or c.isspace() or c in '.,!?' else ' ' for c in text])
    return text


def predict_verdict(text, model, tokenizer):
    """Predict verdict with preprocessing"""
    # Preprocess text
    clean_text = preprocess_text(text)

    # Add special tokens
    text_with_tokens = f"[TITLE] {clean_text}"

    # Tokenize
    inputs = tokenizer(
        text_with_tokens,
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )

    # Move to GPU if available
    if torch.cuda.is_available():
        inputs = {k: v.to("cuda") for k, v in inputs.items()}
        model = model.to("cuda")

    # Get prediction
    with torch.no_grad():
        outputs = model(**inputs)
        probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
        prediction = torch.argmax(probabilities, dim=-1).item()
        confidence = probabilities[0][prediction].item()

    return prediction, confidence

In [ ]:
# https://www.reddit.com/r/AmItheAsshole/comments/1gwl76e/aitah_for_refusing_to_let_my_moms_boyfriend_walk/
text = """AITAH for refusing to let my mom’s boyfriend walk me down the aisle?

I (24F) am getting married next summer, and ever since I started planning the wedding, my mom (48F) has been pushing for her boyfriend (50M) to walk me down the aisle. My dad passed away when I was 10, and my mom started dating her boyfriend about six years ago. While he’s always been nice to me, I’ve never seen him as a father figure, he came into my life when I was already an adult, and we’re friendly but not particularly close.

I told my mom that I plan to walk myself down the aisle as a way to honor my independence and my dad’s memory. She got really upset, saying her boyfriend has “earned” the spot by being there for me all these years. She even accused me of disrespecting her relationship and trying to “erase” my dad, which couldn’t be further from the truth.

Now her boyfriend is avoiding me, and my mom keeps calling me selfish and saying I’m ruining the wedding before it even starts. A few family members are also weighing in, saying I should let him do it to keep the peace. But this is my wedding, and I feel like I should have the final say. AITAH?"""

prediction, confidence = predict_verdict(text, model, tokenizer)
verdict = "NTA" if prediction == 1 else "YTA"
print(f"Verdict: {verdict} (confidence: {confidence:.2f})")

Verdict: NTA (confidence: 0.98)


In [ ]:
# https://www.reddit.com/r/AmItheAsshole/comments/1gw9o5g/aita_for_defending_my_daughters_comments_towards/
text = """AITA for defending my daughters comments towards my other daughter being single?

My daughter (17f, Emily) has been dating this boy (17m, Zach) for around 2 -2 1/2 months now and he recently came to visit us, and this is the first time he has came over for dinner, and this is Emilys first bf. Zach is a very lovely boy and very outgoing. When he came in and saw me he says "Emily, I didn't know you have 2 sisters". I laughed as even though it's cliche I know he's trying to be nice. The entire time at dinner he was very polite but he is also a very outgoing kid. He would say stuff such as what a lovely dinner, this food is great, your backyard is beautiful, etc, etc. So while you could say he was trying to be overly polite, he was still a very sweet and kind kid. Emily is a more shy and reserved person so I felt they were really great for each other. Emily is also very sweet and positive, another thing they have in common that I appreciated. My husband also hit it off with him and they were engaging in sports banter, and eventually came to trash talking some football team owner.

My older daughter (amy,19), however kept grilling the poor guy. Asking if he would pay for dates, to which he said yeah, and then she asks how he has money, and he said his job, then she started talking about making time for Emily, in between school friends and a job. Then it came onto how they would get to dates and she started asking him about his license, she then started to ask about protecting her making comments on his stature (hes on the shorter side and kind of chubby, like 5'7 and maybe a little overweight, nothing crazy however) and he seemed to be getting uncomfortable so I brought out dessert, which he again complimented, and my husband brought up sports to change topics.

After he left I asked her why she would do that. She said that he seemed to nice, and cliche, as if he was faking it. I said so people cant be nice these days? You made it weird for him and Emily, Emily didn't deserve that neither did he. She said that she just didn't like that vibe as no-one is that nice or positive it was definetely forced. Emily butted in and said that she really didnt appreciate that and said that Amy's reasoning didn't make sense. Amy said that she didn't care if it was awkward as she wanted to grill him, and that she doesn't like him because he seemed fake. Emily said, that Amy was messed up and I agreed. Amy then said that he was some dumb weak kid faking being nice, and this upset Emily, and me. Emily then said in a fuss "You only say that because your single and no one will date you". She has been slightly sensitive about this as she hasn't been in a committed relationship yet.This upset Amy and Amy asked why I didn't say anything or stop her from going to her room. I said that she just insulted her bf and that she deserved it, she told me I should punish her and was being a bad parent and now Amy isn't talking to me and I feel that maybe a personal insult like that was to far."""

prediction, confidence = predict_verdict(text, model, tokenizer)
verdict = "NTA" if prediction == 1 else "YTA"
print(f"Verdict: {verdict} (confidence: {confidence:.2f})")

Verdict: YTA (confidence: 0.81)
